In [ ]:
# Generiere aus ...

Tree-Asci eine taxo.json mit Name, Slug, Position in einer 3er Struktur: SuperGroup, Group, Subgroup

In [19]:
# ChatGPT 4.o

import json
from pathlib import Path
from pprint import pprint

input_path = Path("data/2025/taxo.txt")
output_path = Path("data/2025/taxo.tree.json")

def slugify(text):
    return text.strip().lower()\
        .replace(",", "").replace(" & ", " und ")\
        .replace(" ", "-").replace("/", "-")\
        .replace(":", "").replace("(", "").replace(")", "")\
        .replace(".", "").replace("–", "-")

# Initialzähler
super_count = group_count = sub_count = 0

# Finaler Datenbaum
tree = {
    "name": "FOSSGIS25",
    "slug": "fossgis25",
    "supers": []
}

current_super = None
current_group = None

with open(input_path, encoding="utf-8") as f:
    for line in f:
        raw = line.rstrip("\n")
        if not raw.strip() or raw.strip() == "FOSSGIS25/":
            continue

        # Einrückung erkennen
        indent = len(line) - len(line.lstrip())
        depth = indent // 4
        name = raw.strip("├└─│ ").rstrip("/")

        if depth == 1:
            # Supergroup
            super_count += 1
            group_count = 0
            sub_count = 0
            current_super = {
                "name": name,
                "position": f"{super_count:02}|00|00",
                "slug": slugify(name),
                "groups": []
            }
            tree["supers"].append(current_super)

        elif depth == 2:
            # Group
            group_count += 1
            sub_count = 0
            current_group = {
                "name": name,
                "position": f"{super_count:02}|{group_count:02}|00",
                "slug": slugify(name),
                "subs": []
            }
            if current_super is not None:
                current_super["groups"].append(current_group)
            else:
                print(f"⚠️ Group ohne Supergruppe: {name}")

        elif depth == 3:
            # Sub
            sub_count += 1
            sub = {
                "name": name,
                "position": f"{super_count:02}|{group_count:02}|{sub_count:02}",
                "slug": slugify(name)
            }
            if current_group is not None:
                current_group["subs"].append(sub)
            else:
                print(f"⚠️ Sub ohne Gruppe: {name}")

# Save JSON
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(tree, f, ensure_ascii=False, indent=2)

# Ausgabe im Notebook
pprint(tree, sort_dicts=False)


{'name': 'FOSSGIS25',
 'slug': 'fossgis25',
 'supers': [{'name': 'Ethik, Bildung & Community',
             'position': '01|00|00',
             'slug': 'ethik-bildung-und-community',
             'groups': [{'name': 'Digitale Souveränität & Open Source Kultur',
                         'position': '01|01|00',
                         'slug': 'digitale-souveränität-und-open-source-kultur',
                         'subs': []},
                        {'name': 'GIS in Lehre, Schule & Hochschule',
                         'position': '01|02|00',
                         'slug': 'gis-in-lehre-schule-und-hochschule',
                         'subs': []},
                        {'name': 'Community Building & Diversität',
                         'position': '01|03|00',
                         'slug': 'community-building-und-diversität',
                         'subs': []}]}]}


In [10]:
import json
import re

def ascii_tree_to_json(ascii_tree):
    lines = ascii_tree.strip().split('\n')
    if not lines:
        return {}
    
    root_line = lines[0].strip()
    root = {'name': root_line, 'children': []}
    stack = [(0, root)]  # (depth, node)
    
    for line in lines[1:]:
        line = line.rstrip()
        if not line:
            continue
        
        # Finde die letzte "── " zur Trennung von Prefix und Namen
        idx = line.rfind("── ")
        if idx == -1:
            continue  # Ignoriere ungültige Zeilen
        
        prefix = line[:idx]
        node_name = line[idx+3:].strip()
        
        # Berechne Tiefe durch Zählen der "│   " im Prefix
        depth = len(re.findall(r'│   ', prefix)) + 1
        
        new_node = {'name': node_name, 'children': []}
        
        # Finde den passenden Elternknoten
        while stack and stack[-1][0] >= depth:
            stack.pop()
        if stack:
            parent = stack[-1][1]
            parent['children'].append(new_node)
        
        stack.append((depth, new_node))
    
    return root

# Beispielnutzung
ascii_tree = """
FOSSGIS25/
├── Tools & Technologie/
│   ├── GIS & Open-Source-Software
│   │   ├── QGIS, GRASS, SAGA
│   │   └── Desktop- & Mobile-GIS
│   ├── Geodateninfrastrukturen & Standards
│   │   ├── INSPIRE, WMS/WFS/WCS
│   │   └── Metadaten & Geoportale
│   ├── Automatisierung & Scripting
│   │   ├── Python in QGIS
│   │   ├── Geo-ETL (ogr2ogr, n8n, Node-RED)
│   │   └── Datenpipelines & APIs
│   └── Webmapping & Visualisierung
│       ├── Leaflet, MapLibre, OpenLayers
│       ├── Kartenserver (MapServer, GeoServer)
│       └── UI/UX & 3D-Webmaps
├── Daten & Quellen/
│   ├── OpenStreetMap & Community
│   │   ├── Mapping-Tools (JOSM, iD, StreetComplete)
│   │   ├── Qualitätssicherung
│   │   └── Community-Projekte & OSM-basierte Dienste
│   ├── Open Data & Verwaltungsdaten
│   │   ├── Kommunale Datenportale
│   │   ├── Lizenzen & Nutzbarkeit
│   │   └── Bürgerbeteiligung
│   ├── Fernerkundung & Bildverarbeitung
│   │   ├── Satellitendaten (Sentinel, Landsat)
│   │   ├── Drohnen & Photogrammetrie
│   │   └── Lidar & Punktwolken
│   └── Geodatenmanagement & Datenbanken
│       ├── PostGIS, SpatiaLite, GeoPackage
│       ├── ETL-Prozesse & Formatkonvertierung
│       └── Datenqualität & Replikation
├── Anwendung & Gestaltung/
│   ├── Kartographie & Storytelling
│   │   ├── Map Design & Typographie
│   │   ├── Thematische Karten
│   │   └── StoryMaps & Data Journalism
│   └── 3D, VR/AR & immersives Mapping
│       ├── CesiumJS, A-Frame, WebXR
│       └── 3D-Stadtmodelle & virtuelle Zwillinge
└── Wissen & Gesellschaft/
    └── Ethik, Bildung & Community
        ├── Digitale Souveränität & Open Source Kultur
        ├── GIS in Lehre, Schule & Hochschule
        └── Community Building & Diversität
"""

json_tree = ascii_tree_to_json(ascii_tree)
print(json.dumps(json_tree, indent=4, ensure_ascii=False))

{
    "name": "FOSSGIS25/",
    "children": [
        {
            "name": "Tools & Technologie/",
            "children": [
                {
                    "name": "GIS & Open-Source-Software",
                    "children": [
                        {
                            "name": "QGIS, GRASS, SAGA",
                            "children": []
                        },
                        {
                            "name": "Desktop- & Mobile-GIS",
                            "children": []
                        }
                    ]
                },
                {
                    "name": "Geodateninfrastrukturen & Standards",
                    "children": [
                        {
                            "name": "INSPIRE, WMS/WFS/WCS",
                            "children": []
                        },
                        {
                            "name": "Metadaten & Geoportale",
                            "children": []
        

In [18]:
import json
import re # Regular expressions can help clean the names

def parse_ascii_tree_visual(ascii_string):
    """
    Konvertiert einen ASCII-Baum mit visuellen Verbindern (├──, └──, │)
    und 4 Leerzeichen pro Ebene in ein verschachteltes JSON-Objekt.
    Jeder Knoten wird als Dictionary {'name': 'NodeName', 'children': []} dargestellt.
    Entfernt ein optionales '/' am Ende des Namens.
    """
    lines = ascii_string.strip().split('\n')
    if not lines or not lines[0].strip():
        return None

    parent_stack = [] # Stack: (ebene, knoten_dict)
    root_nodes = []   # Liste für Knoten auf Ebene 0

    for line in lines:
        stripped_line_for_indent = line.lstrip(' ')
        if not stripped_line_for_indent:
            continue # Überspringe leere Zeilen

        indentation = len(line) - len(stripped_line_for_indent)
        level = indentation // 4 # Ebene basierend auf 4 Leerzeichen

        # Namen extrahieren und von Baumzeichen säubern
        # 1. Entferne die Einrückung
        content_part = line.lstrip(' ')
        # 2. Entferne die Baum-Präfixe (│, ├──, └── und Leerzeichen danach)
        #    Regex sucht nach optionalen '│ ' am Anfang, gefolgt von '├── ' oder '└── '
        #    oder matcht einfach den gesamten String, wenn kein Connector da ist (für Root)
        match = re.match(r'^[│\s]*(?:[├└]──\s)?(.*)', content_part)
        if match:
            node_name = match.group(1)
        else:
            # Fallback oder Root-Element
            node_name = content_part

        # 3. Bereinige den Namen (Leerzeichen, / am Ende)
        node_name = node_name.strip().rstrip('/')
        if not node_name: # Falls nach Bereinigung nichts übrig bleibt
             continue

        # Erstelle das neue Knoten-Dictionary
        new_node = {"name": node_name, "children": []}

        # Finde den richtigen Elternknoten im Stack
        while parent_stack and parent_stack[-1][0] >= level:
            parent_stack.pop()

        # Füge den neuen Knoten an der richtigen Stelle ein
        if not parent_stack:
            # Ebene 0 -> Wurzelknoten
            root_nodes.append(new_node)
        else:
            # Kind eines Knotens im Stack
            parent_node = parent_stack[-1][1]
            parent_node["children"].append(new_node)

        # Füge den aktuellen Knoten zum Stack hinzu
        parent_stack.append((level, new_node))

    # Rückgabe anpassen (einzelner Root vs. mehrere)
    if len(root_nodes) == 1:
        return root_nodes[0]
    elif len(root_nodes) > 1:
         # Wenn es mehrere "Wurzeln" gibt, packen wir sie in ein gemeinsames Root-Objekt
         # oder geben die Liste zurück, je nach Bedarf. Hier: Liste.
        return root_nodes
    else:
        return None # Kein gültiger Baum gefunden


# --- Dein ASCII-Tree ---
ascii_tree = """
FOSSGIS25/
├── Tools & Technologie/
│   ├── GIS & Open-Source-Software
│   │   ├── QGIS, GRASS, SAGA
│   │   └── Desktop- & Mobile-GIS
│   ├── Geodateninfrastrukturen & Standards
│   │   ├── INSPIRE, WMS/WFS/WCS
│   │   └── Metadaten & Geoportale
│   ├── Automatisierung & Scripting
│   │   ├── Python in QGIS
│   │   ├── Geo-ETL (ogr2ogr, n8n, Node-RED)
│   │   └── Datenpipelines & APIs
│   └── Webmapping & Visualisierung
│       ├── Leaflet, MapLibre, OpenLayers
│       ├── Kartenserver (MapServer, GeoServer)
│       └── UI/UX & 3D-Webmaps
│
├── Daten & Quellen/
│   ├── OpenStreetMap & Community
│   │   ├── Mapping-Tools (JOSM, iD, StreetComplete)
│   │   ├── Qualitätssicherung
│   │   └── Community-Projekte & OSM-basierte Dienste
│   ├── Open Data & Verwaltungsdaten
│   │   ├── Kommunale Datenportale
│   │   ├── Lizenzen & Nutzbarkeit
│   │   └── Bürgerbeteiligung
│   ├── Fernerkundung & Bildverarbeitung
│   │   ├── Satellitendaten (Sentinel, Landsat)
│   │   ├── Drohnen & Photogrammetrie
│   │   └── Lidar & Punktwolken
│   └── Geodatenmanagement & Datenbanken
│       ├── PostGIS, SpatiaLite, GeoPackage
│       ├── ETL-Prozesse & Formatkonvertierung
│       └── Datenqualität & Replikation
│
├── Anwendung & Gestaltung/
│   ├── Kartographie & Storytelling
│   │   ├── Map Design & Typographie
│   │   ├── Thematische Karten
│   │   └── StoryMaps & Data Journalism
│   └── 3D, VR/AR & immersives Mapping
│       ├── CesiumJS, A-Frame, WebXR
│       └── 3D-Stadtmodelle & virtuelle Zwillinge
│
└── Wissen & Gesellschaft/
    └── Ethik, Bildung & Community
        ├── Digitale Souveränität & Open Source Kultur
        ├── GIS in Lehre, Schule & Hochschule
        └── Community Building & Diversität
"""

# Konvertiere den Baum
json_structure = parse_ascii_tree_visual(ascii_tree)

# Gib das Ergebnis als schönen JSON-String aus
if json_structure:
    print(json.dumps(json_structure, indent=2, ensure_ascii=False)) # indent=2 für kompaktere Ausgabe, ensure_ascii=False für Umlaute etc.
else:
    print("Konnte den Baum nicht parsen.")

[
  {
    "name": "FOSSGIS25",
    "children": []
  },
  {
    "name": "Tools & Technologie",
    "children": []
  },
  {
    "name": "GIS & Open-Source-Software",
    "children": []
  },
  {
    "name": "QGIS, GRASS, SAGA",
    "children": []
  },
  {
    "name": "Desktop- & Mobile-GIS",
    "children": []
  },
  {
    "name": "Geodateninfrastrukturen & Standards",
    "children": []
  },
  {
    "name": "INSPIRE, WMS/WFS/WCS",
    "children": []
  },
  {
    "name": "Metadaten & Geoportale",
    "children": []
  },
  {
    "name": "Automatisierung & Scripting",
    "children": []
  },
  {
    "name": "Python in QGIS",
    "children": []
  },
  {
    "name": "Geo-ETL (ogr2ogr, n8n, Node-RED)",
    "children": []
  },
  {
    "name": "Datenpipelines & APIs",
    "children": []
  },
  {
    "name": "Webmapping & Visualisierung",
    "children": []
  },
  {
    "name": "Leaflet, MapLibre, OpenLayers",
    "children": []
  },
  {
    "name": "Kartenserver (MapServer, GeoServer)",
    "chi

In [13]:
# Googles erster Versuch

import json
import re

def parse_ascii_tree_structured_corrected(ascii_string):
    """
    Konvertiert einen ASCII-Baum (4 Leerzeichen pro Ebene) in eine spezifische
    verschachtelte JSON-Struktur: root > supergroups > groups > subgroups.
    Behandelt den ersten Knoten als Root und ordnet nachfolgende Ebenen korrekt zu.
    Entfernt visuelle Verbinder und ein optionales '/' am Ende des Namens.
    """
    lines = ascii_string.strip().split('\n')
    if not lines or not lines[0].strip():
        return None

    root_node = None
    parent_stack = [] # Stack: (level, node_dict)

    for line_num, line in enumerate(lines):
        stripped_line_for_indent = line.lstrip(' ')
        if not stripped_line_for_indent:
            continue

        indentation = len(line) - len(stripped_line_for_indent)
        level = indentation // 4

        # Namen extrahieren und säubern
        content_part = line.lstrip(' ')
        match = re.match(r'^[│\s]*(?:[├└]──\s)?(.*)', content_part)
        if match:
            node_name = match.group(1)
        else:
            node_name = content_part
        node_name = node_name.strip().rstrip('/')
        if not node_name:
             continue

        # Erstelle den Knoten basierend auf seiner EIGENEN Ebene
        # Die Zuweisung zum Parent erfolgt später basierend auf der PARENT-Ebene
        if level == 0:
            # Der Root-Knoten enthält 'supergroups'
            new_node = {"name": node_name, "supergroups": []}
            # WICHTIG: Nur den allerersten Knoten als root_node setzen
            if root_node is None:
                 root_node = new_node
            else:
                 # Fehler: Mehrere Knoten auf Ebene 0 gefunden, das sollte nicht sein
                 print(f"Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere '{node_name}'.")
                 continue # Ignoriere diesen Knoten

        elif level == 1:
            # Ein Knoten auf Ebene 1 (Supergroup) enthält 'groups'
            new_node = {"name": node_name, "groups": []}
        elif level == 2:
            # Ein Knoten auf Ebene 2 (Group) enthält 'subgroups'
            new_node = {"name": node_name, "subgroups": []}
        else: # level >= 3
            # Ein Knoten auf Ebene 3+ (Subgroup) enthält auch 'subgroups'
            # (oder ist ein Blattknoten, aber wir halten die Struktur konsistent)
            new_node = {"name": node_name, "subgroups": []}
            # Alternative für Blattknoten:
            # new_node = {"name": node_name}

        # --- Korrekte Elternfindung und Verknüpfung ---

        # 1. Stack anpassen: Alle Knoten entfernen, die auf gleicher oder tieferer Ebene liegen
        while parent_stack and parent_stack[-1][0] >= level:
            parent_stack.pop()

        # 2. Verknüpfen: Wenn ein Elternteil im Stack ist...
        if parent_stack:
            parent_level, parent_node = parent_stack[-1]

            # Füge 'new_node' zur korrekten Liste des 'parent_node' hinzu,
            # basierend auf der Ebene des *Elternteils* (parent_level)
            if parent_level == 0:
                parent_node['supergroups'].append(new_node)
            elif parent_level == 1:
                parent_node['groups'].append(new_node)
            elif parent_level >= 2:
                 # Stelle sicher, dass der Parent eine 'subgroups'-Liste hat
                 # (Sollte durch Node-Erstellung oben immer der Fall sein)
                 if 'subgroups' not in parent_node:
                      parent_node['subgroups'] = [] # Nur zur Sicherheit
                 parent_node['subgroups'].append(new_node)

        # 3. Fehlerprüfung (optional aber gut): Knoten ohne Parent, der nicht Root ist?
        elif level != 0 and root_node is not None:
             print(f"Warnung: Knoten '{node_name}' auf Ebene {level} hat keinen Parent im Stack.")
             # Hier könnte man entscheiden, ob man ihn ignoriert oder woanders einfügt.
             # In einem korrekt formatierten Baum sollte dies nicht passieren,
             # außer direkt nach dem Root (Level 0), was oben behandelt wird.

        # 4. Aktuellen Knoten auf den Stack legen, damit er Parent sein kann
        #    Stelle sicher, dass auch der Root auf den Stack kommt
        if level == 0 and root_node is not None and not parent_stack: # Nur den echten Root beim ersten Mal
            parent_stack.append((level, new_node))
        elif parent_stack: # Nur wenn es einen Parent gibt (oder wenn es der Root war)
             parent_stack.append((level, new_node))
        # Diese Logik stellt sicher, dass der Knoten nur auf den Stack kommt,
        # wenn er Teil des gültigen Baumes ist (also entweder Root ist oder einen Parent hat)
        # Korrektur: Einfacher ist es, den Knoten *immer* auf den Stack zu legen,
        # nachdem er korrekt verknüpft wurde (oder als Root identifiziert).
        parent_stack.append((level, new_node)) # <-- Einfachere Logik hier

    return root_node

# --- Dein ASCII-Tree ---
ascii_tree = """
FOSSGIS25/
├── Tools & Technologie/
│   ├── GIS & Open-Source-Software
│   │   ├── QGIS, GRASS, SAGA
│   │   └── Desktop- & Mobile-GIS
│   ├── Geodateninfrastrukturen & Standards
│   │   ├── INSPIRE, WMS/WFS/WCS
│   │   └── Metadaten & Geoportale
│   ├── Automatisierung & Scripting
│   │   ├── Python in QGIS
│   │   ├── Geo-ETL (ogr2ogr, n8n, Node-RED)
│   │   └── Datenpipelines & APIs
│   └── Webmapping & Visualisierung
│       ├── Leaflet, MapLibre, OpenLayers
│       ├── Kartenserver (MapServer, GeoServer)
│       └── UI/UX & 3D-Webmaps
│
├── Daten & Quellen/
│   ├── OpenStreetMap & Community
│   │   ├── Mapping-Tools (JOSM, iD, StreetComplete)
│   │   ├── Qualitätssicherung
│   │   └── Community-Projekte & OSM-basierte Dienste
│   ├── Open Data & Verwaltungsdaten
│   │   ├── Kommunale Datenportale
│   │   ├── Lizenzen & Nutzbarkeit
│   │   └── Bürgerbeteiligung
│   ├── Fernerkundung & Bildverarbeitung
│   │   ├── Satellitendaten (Sentinel, Landsat)
│   │   ├── Drohnen & Photogrammetrie
│   │   └── Lidar & Punktwolken
│   └── Geodatenmanagement & Datenbanken
│       ├── PostGIS, SpatiaLite, GeoPackage
│       ├── ETL-Prozesse & Formatkonvertierung
│       └── Datenqualität & Replikation
│
├── Anwendung & Gestaltung/
│   ├── Kartographie & Storytelling
│   │   ├── Map Design & Typographie
│   │   ├── Thematische Karten
│   │   └── StoryMaps & Data Journalism
│   └── 3D, VR/AR & immersives Mapping
│       ├── CesiumJS, A-Frame, WebXR
│       └── 3D-Stadtmodelle & virtuelle Zwillinge
│
└── Wissen & Gesellschaft/
    └── Ethik, Bildung & Community
        ├── Digitale Souveränität & Open Source Kultur
        ├── GIS in Lehre, Schule & Hochschule
        └── Community Building & Diversität
"""

# Konvertiere den Baum
json_structure = parse_ascii_tree_structured_corrected(ascii_tree)

# Gib das Ergebnis als schönen JSON-String aus
if json_structure:
    print(json.dumps(json_structure, indent=2, ensure_ascii=False))
else:
    print("Konnte den Baum nicht parsen.")

Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere 'Tools & Technologie'.
Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere 'GIS & Open-Source-Software'.
Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere 'QGIS, GRASS, SAGA'.
Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere 'Desktop- & Mobile-GIS'.
Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere 'Geodateninfrastrukturen & Standards'.
Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere 'INSPIRE, WMS/WFS/WCS'.
Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere 'Metadaten & Geoportale'.
Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere 'Automatisierung & Scripting'.
Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere 'Python in QGIS'.
Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere 'Geo-ETL (ogr2ogr, n8n, Node-RED)'.
Warnung: Mehrere Root-Knoten (Level 0) gefunden. Ignoriere 'Datenpipelines & APIs'.
Warnung: Mehrere Root-Knoten (Level 0) gefunden. Igno

In [14]:
# DeepSeek erster File-Schreiber

import json
import re
import os

def generate_slug(name):
    slug = name.lower()
    replacements = {'ä':'ae', 'ö':'oe', 'ü':'ue', 'ß':'ss'}
    for umlaut, replacement in replacements.items():
        slug = slug.replace(umlaut, replacement)
    slug = re.sub(r'[^a-z0-9/]+', '-', slug)
    slug = re.sub(r'-+', '-', slug).strip('-')
    slug = re.sub(r'/-', '/', slug)
    return slug

def ascii_tree_to_json(ascii_tree):
    lines = ascii_tree.strip().split('\n')
    if not lines:
        return {}
    
    root_line = lines[0].strip()
    root = {'name': root_line, 'slug': generate_slug(root_line), 'children': []}
    stack = [(0, root)]

    for line in lines[1:]:
        line = line.rstrip()
        if not line:
            continue
        
        idx = line.rfind("── ")
        if idx == -1:
            continue

        prefix = line[:idx]
        node_name = line[idx+3:].strip()
        depth = len(re.findall(r'│   ', prefix)) + 1

        new_node = {
            'name': node_name,
            'slug': generate_slug(node_name),
            'children': []
        }

        while stack and stack[-1][0] >= depth:
            stack.pop()
        if stack:
            parent = stack[-1][1]
            parent['children'].append(new_node)

        stack.append((depth, new_node))
    
    return root

# Dateipfade
input_path = "data/2025/taxo.txt"
output_path = "data/2025/taxo.json"

# Verzeichnisse erstellen falls nötig
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Verarbeitung
with open(input_path, 'r', encoding='utf-8') as f:
    ascii_tree = f.read()

json_tree = ascii_tree_to_json(ascii_tree)

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(json_tree, f, indent=4, ensure_ascii=False)

In [17]:
# Google 2ter Versuch

import json
import re

def parse_ascii_tree_structured_final(ascii_string):
    """
    Konvertiert einen ASCII-Baum (4 Leerzeichen pro Ebene) in eine spezifische
    verschachtelte JSON-Struktur: root > supergroups > groups > subgroups.
    Verwendet eine robustere Methode zur Bestimmung der Einrückungsebene.
    Entfernt visuelle Verbinder und ein optionales '/' am Ende des Namens.
    """
    lines = ascii_string.strip().split('\n')
    if not lines:
        return None

    root_node = None
    parent_stack = [] # Stack: (level, node_dict)

    for line_num, line in enumerate(lines):
        # Überspringe komplett leere oder nur aus Whitespace bestehende Zeilen
        if not line.strip():
            continue

        # --- Robuste Einrückungs- und Ebenenbestimmung ---
        indentation = 0
        # Finde den Index des Startzeichens des Verbinders '├──' or '└──'
        connector_match = re.search(r'[├└]', line)
        if connector_match:
            indentation = connector_match.start()
        else:
            # Kein Verbinder gefunden -> Muss Root sein oder ein Knoten ohne Verbinder?
            # Finde den Index des ersten Nicht-Leerzeichens
            first_char_match = re.search(r'\S', line)
            if first_char_match:
                indentation = first_char_match.start()
            else:
                # Sollte nicht passieren, da leere Zeilen oben abgefangen werden
                continue

        level = indentation // 4 # Ebene basierend auf 4 Leerzeichen pro Einrückung

        # --- Namen extrahieren und säubern ---
        # Entferne zuerst die Einrückung und dann die Verbinder/Pipes
        content_part = line[indentation:] # Teil ab der Einrückung
        # Regex, um führende Pipes/Leerzeichen und Verbinder zu entfernen
        name_match = re.match(r'^[│\s]*(?:[├└]──\s)?(.*)', content_part)
        if name_match:
            node_name = name_match.group(1).strip().rstrip('/')
        else:
            # Fallback, wenn Regex nicht passt (sollte für Root der Fall sein)
            node_name = content_part.strip().rstrip('/')

        if not node_name:
             # Passiert, wenn eine Zeile nur aus Einrückung/Struktur besteht
             continue

        # --- Debugging-Ausgabe (optional, zum Nachvollziehen) ---
        # print(f"Line: '{line}' | Indent: {indentation} | Level: {level} | Name: '{node_name}'")


        # --- Strukturierte Node-Erstellung ---
        if level == 0:
            new_node = {"name": node_name, "supergroups": []}
            if root_node is None:
                 root_node = new_node
            else:
                 # Diese Warnung sollte jetzt nicht mehr fälschlicherweise erscheinen
                 print(f"FEHLER: Mehrere Root-Knoten (Level {level}) gefunden. Überspringe '{node_name}'. Zeile: '{line}'")
                 continue
        elif level == 1:
            new_node = {"name": node_name, "groups": []}
        elif level == 2:
            new_node = {"name": node_name, "subgroups": []}
        else: # level >= 3
            new_node = {"name": node_name, "subgroups": []}

        # --- Stack anpassen ---
        # Entferne Eltern vom Stack, deren Ebene >= der aktuellen Ebene ist
        while parent_stack and parent_stack[-1][0] >= level:
            parent_stack.pop()

        # --- Strukturierte Verknüpfung ---
        if parent_stack:
            parent_level, parent_node = parent_stack[-1]
            if parent_level == 0:
                # setdefault ist sicherer, falls die Liste mal fehlt
                parent_node.setdefault('supergroups', []).append(new_node)
            elif parent_level == 1:
                parent_node.setdefault('groups', []).append(new_node)
            elif parent_level >= 2:
                parent_node.setdefault('subgroups', []).append(new_node)
        elif level != 0 and root_node is not None:
             # Ein Knoten > Level 0 muss einen Parent haben (nachdem der Root erstellt wurde)
             print(f"WARNUNG: Knoten '{node_name}' (Level {level}) hat keinen Parent im Stack. Zeile: '{line}'")
             # Evtl. den Knoten ignorieren oder anders behandeln?
             continue # Ignorieren wir ihn vorerst

        # --- Aktuellen Knoten zum Stack hinzufügen ---
        parent_stack.append((level, new_node))

    # --- Aufräumen: Entferne leere Listen am Ende (optional) ---
    def cleanup_empty_lists(node):
        if "supergroups" in node and not node["supergroups"]: del node["supergroups"]
        if "groups" in node and not node["groups"]: del node["groups"]
        if "subgroups" in node and not node["subgroups"]: del node["subgroups"]

        for key in ["supergroups", "groups", "subgroups"]:
             if key in node:
                 for child in node[key]:
                     cleanup_empty_lists(child)

    # if root_node:
    #      cleanup_empty_lists(root_node) # Optionales Cleanup aktivieren/deaktivieren

    return root_node

# --- Dein ASCII-Tree ---
# (Unverändert)
ascii_tree = """
FOSSGIS25/
├── Tools & Technologie/
│   ├── GIS & Open-Source-Software
│   │   ├── QGIS, GRASS, SAGA
│   │   └── Desktop- & Mobile-GIS
│   ├── Geodateninfrastrukturen & Standards
│   │   ├── INSPIRE, WMS/WFS/WCS
│   │   └── Metadaten & Geoportale
│   ├── Automatisierung & Scripting
│   │   ├── Python in QGIS
│   │   ├── Geo-ETL (ogr2ogr, n8n, Node-RED)
│   │   └── Datenpipelines & APIs
│   └── Webmapping & Visualisierung
│       ├── Leaflet, MapLibre, OpenLayers
│       ├── Kartenserver (MapServer, GeoServer)
│       └── UI/UX & 3D-Webmaps
│
├── Daten & Quellen/
│   ├── OpenStreetMap & Community
│   │   ├── Mapping-Tools (JOSM, iD, StreetComplete)
│   │   ├── Qualitätssicherung
│   │   └── Community-Projekte & OSM-basierte Dienste
│   ├── Open Data & Verwaltungsdaten
│   │   ├── Kommunale Datenportale
│   │   ├── Lizenzen & Nutzbarkeit
│   │   └── Bürgerbeteiligung
│   ├── Fernerkundung & Bildverarbeitung
│   │   ├── Satellitendaten (Sentinel, Landsat)
│   │   ├── Drohnen & Photogrammetrie
│   │   └── Lidar & Punktwolken
│   └── Geodatenmanagement & Datenbanken
│       ├── PostGIS, SpatiaLite, GeoPackage
│       ├── ETL-Prozesse & Formatkonvertierung
│       └── Datenqualität & Replikation
│
├── Anwendung & Gestaltung/
│   ├── Kartographie & Storytelling
│   │   ├── Map Design & Typographie
│   │   ├── Thematische Karten
│   │   └── StoryMaps & Data Journalism
│   └── 3D, VR/AR & immersives Mapping
│       ├── CesiumJS, A-Frame, WebXR
│       └── 3D-Stadtmodelle & virtuelle Zwillinge
│
└── Wissen & Gesellschaft/
    └── Ethik, Bildung & Community
        ├── Digitale Souveränität & Open Source Kultur
        ├── GIS in Lehre, Schule & Hochschule
        └── Community Building & Diversität
"""

# Konvertiere den Baum
json_structure = parse_ascii_tree_structured_final(ascii_tree)

# Gib das Ergebnis als schönen JSON-String aus
if json_structure:
    print(json.dumps(json_structure, indent=2, ensure_ascii=False))
else:
    print("Konnte den Baum nicht parsen oder Baum war leer.")

FEHLER: Mehrere Root-Knoten (Level 0) gefunden. Überspringe 'Tools & Technologie'. Zeile: '├── Tools & Technologie/'
FEHLER: Mehrere Root-Knoten (Level 0) gefunden. Überspringe 'Daten & Quellen'. Zeile: '├── Daten & Quellen/'
FEHLER: Mehrere Root-Knoten (Level 0) gefunden. Überspringe 'Anwendung & Gestaltung'. Zeile: '├── Anwendung & Gestaltung/'
FEHLER: Mehrere Root-Knoten (Level 0) gefunden. Überspringe 'Wissen & Gesellschaft'. Zeile: '└── Wissen & Gesellschaft/'
{
  "name": "FOSSGIS25",
  "supergroups": [
    {
      "name": "GIS & Open-Source-Software",
      "groups": [
        {
          "name": "QGIS, GRASS, SAGA",
          "subgroups": []
        },
        {
          "name": "Desktop- & Mobile-GIS",
          "subgroups": []
        }
      ]
    },
    {
      "name": "Geodateninfrastrukturen & Standards",
      "groups": [
        {
          "name": "INSPIRE, WMS/WFS/WCS",
          "subgroups": []
        },
        {
          "name": "Metadaten & Geoportale",
        

In [22]:
# DeepSeek inkl. Position

import json
import re
import os

def generate_slug(name):
    slug = name.lower()
    replacements = {'ä':'ae', 'ö':'oe', 'ü':'ue', 'ß':'ss'}
    for umlaut, repl in replacements.items():
        slug = slug.replace(umlaut, repl)
    slug = re.sub(r'[^a-z0-9/]+', '-', slug)
    slug = re.sub(r'-+', '-', slug).strip('-')
    slug = re.sub(r'/-', '/', slug)
    return slug

def ascii_tree_to_json(ascii_tree):
    lines = ascii_tree.strip().split('\n')
    root_line = lines[0].strip()
    root = {'name': root_line, 'slug': generate_slug(root_line), 'position': '', 'children': []}
    stack = [(0, root)]  # (depth, node)

    for line in lines[1:]:
        line = line.rstrip()
        if not line or "── " not in line:
            continue

        # Parse Line
        prefix, node_name = line.rsplit("── ", 1)
        node_name = node_name.strip()
        depth = prefix.count("│   ") + 1

        # Find Parent
        while stack and stack[-1][0] >= depth:
            stack.pop()
        parent = stack[-1][1] if stack else None

        if parent:
            # Calculate 3-Part Position
            child_depth = depth
            part_index = child_depth - 1
            parent_parts = parent['position'].split('|') if parent['position'] else ['00', '00', '00']
            parent_parts = (parent_parts + ['00']*3)[:3]  # Ensure 3 parts
            
            child_index = len(parent['children']) + 1
            new_parts = [
                f"{child_index:02d}" if i == part_index else '00' if i > part_index else parent_parts[i]
                for i in range(3)
            ]
            
            new_node = {
                'name': node_name,
                'slug': generate_slug(node_name),
                'position': '|'.join(new_parts),
                'children': []
            }
            parent['children'].append(new_node)
            stack.append((depth, new_node))

    return root

# Dateihandling
input_path = "data/2025/taxo.txt"
output_path = "data/2025/taxo.json"

os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(input_path, 'r', encoding='utf-8') as f:
    ascii_tree = f.read()

json_tree = ascii_tree_to_json(ascii_tree)

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(json_tree, f, indent=2, ensure_ascii=False)

In [24]:
# DeepSeek Korrektur 1

import json
import re
import os

def generate_slug(name):
    slug = name.lower()
    replacements = {'ä':'ae', 'ö':'oe', 'ü':'ue', 'ß':'ss'}
    for umlaut, repl in replacements.items():
        slug = slug.replace(umlaut, repl)
    slug = re.sub(r'[^a-z0-9/]+', '-', slug)
    slug = re.sub(r'-+', '-', slug).strip('-')
    slug = re.sub(r'/-', '/', slug)
    return slug

def ascii_tree_to_json(ascii_tree):
    lines = ascii_tree.strip().split('\n')
    if not lines:
        return {}
    
    root_line = lines[0].strip()
    root = {'name': root_line, 'slug': generate_slug(root_line), 'position': '', 'children': []}
    stack = [(0, root)]  # (depth, node)

    for line in lines[1:]:
        line = line.rstrip()
        if not line or "── " not in line:
            continue

        # Tiefenberechnung über Einrückung (4 Leerzeichen = 1 Ebene)
        indent = len(line) - len(line.lstrip(' '))
        depth = (indent // 4) + 1
        
        # Trenne Prefix und Knotennamen
        prefix, node_name = line.rsplit("── ", 1)
        node_name = node_name.strip()

        # Finde Elternelement im Stack
        while stack and stack[-1][0] >= depth:
            stack.pop()
        parent = stack[-1][1] if stack else None

        if parent:
            # Berechne 3-teilige Position
            child_index = len(parent['children']) + 1
            if parent['position']:
                parent_parts = parent['position'].split('|')
                new_parts = [
                    parent_parts[0] if depth == 2 else '00',
                    f"{child_index:02d}" if depth == 2 else parent_parts[1] if depth == 3 else '00',
                    f"{child_index:02d}" if depth == 3 else '00'
                ]
                new_position = '|'.join(new_parts)
            else:
                new_position = f"{child_index:02d}|00|00"

            new_node = {
                'name': node_name,
                'slug': generate_slug(node_name),
                'position': new_position,
                'children': []
            }
            parent['children'].append(new_node)
            stack.append((depth, new_node))

    return root

# Dateipfade
input_path = "data/2025/taxo.txt"
output_path = "data/2025/taxo.json"

# Verzeichnisse erstellen falls nötig
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Verarbeitung
with open(input_path, 'r', encoding='utf-8') as f:
    ascii_tree = f.read()

json_tree = ascii_tree_to_json(ascii_tree)

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(json_tree, f, indent=2, ensure_ascii=False)